# Test Code for Experiment Speed

In [1]:
import os
import json

import driverlessai

In [2]:
driverlessai.__version__

'1.10.1.3'

In [3]:
# Driverless AIのuser nameとpasswordの読み込み
with open(os.path.join('..', 'idpass.json')) as f:
    idpass = json.load(f)

### 実験実施用関数

In [4]:
def get_dai_client(daiaddress, daipassword) -> 'driverlessai._core.Client':
    '''
    DAIサーバへの接続
    ----------
    daiaddress : str
    daipassword : str
    '''
    print('----- start server connection : get_dai_client -----')
    # Driverless AIサーバーへの接続
    dai = driverlessai.Client(address=daiaddress, username=idpass['id'], password=daipassword)
    return dai

def get_dataset(daiobj, dataname, dataurl) -> 'driverlessai._datasets.Dataset': 
    '''
    データオブジェクトの取得
    ----------
    daiobj : driverlessai._core.Client
    dataname : str
    dataurl : str
    '''
    print('----- start get data : get_dataset -----')
    # DAI上のデータ一覧
    uploaded_data = {i.name:i.key for i in daiobj.datasets.list()}
    print('Uploaded data name : key >> ', uploaded_data)

    # データ取得
    if dataname in uploaded_data.keys():
        print('Data is already uploaded in DAI')
        ds = daiobj.datasets.get(uploaded_data[data_name]) 
    else:
        print('Data is uploading to DAI.')
        ds = daiobj.datasets.create(data=dataurl, data_source='s3')
    
    return ds

def get_experiment(daiobj, dataobj, target_column, task, drop_columns, enable_gpus) -> 'driverlessai._experiments.Experiment':
    '''
    Experimentの実行とExperimentオブジェクトの取得
    ----------
    daiobj : driverlessai._core.Client
    dataobj : driverlessai._datasets.Dataset
    target_column : str
    task : str
    drop_columns : List[str]
    enable_gpus : bool
    '''
    print('----- start experiment : get_experiment -----')
    # Experiment設定
    dai_settings = {
        'train_dataset': dataobj, 
        'target_column': target_column,
        'task': task,
        'drop_columns': drop_columns,
        'enable_gpus': enable_gpus
    }
    # Experimentの実行
    ex = daiobj.experiments.create(**dai_settings)
    return ex

### 実験のパラメータ情報

In [31]:
# Driverless AIサーバー情報
dai_address = 'http://3.82.142.224'
dai_password = idpass['pass11013']
# データ情報
data_name = 'BostonHousing.csv'
s3url = 's3://h2oai-jp-public/sample_data/boston_house/BostonHousing.csv'   # DAIにアップされてない場合の取得先S3
# Experiment設定
target_column = 'MEDV'
task = 'regression'    # 'regression', 'classification', or 'unsupervised'
drop_columns = []    # List[str] 説明変数として利用しないカラム
#drop_columns = None
enable_gpus = False

### 実験

In [32]:
print('*************** DAIへ接続 ***************')
dai = get_dai_client(daiaddress=dai_address, daipassword=dai_password)
print(type(dai))
print('DAIバージョン: {}'.format(dai.server.version))


print('*************** データの取得 ***************')
ds = get_dataset(daiobj=dai, dataname=data_name, dataurl=s3url) 

print(type(ds))
print('Dataサイズ(byte): {}'.format(ds.file_size))
print('Dataサイズ(mega byte): {}'.format(ds.file_size/1024**2))
print('Data shape: {}'.format(ds.shape))


print('*************** Experimentの実施 ***************')
ex = get_experiment(daiobj=dai, dataobj=ds, 
                    target_column=target_column, task=task, drop_columns=drop_columns, enable_gpus=enable_gpus)

print(type(ds))
print('学習時間（sec）：{}'.format(ex.run_duration))
print('学習時間（min）：{}'.format(ex.run_duration/60))
print('Experimentサイズ（byte）：{}'.format(ex.size))
print('Experimentサイズ（mega byte）：{}'.format(ex.size/1024**2))
print('********** Experiment Summary **********')
ex.summary()

*************** DAIへ接続 ***************
----- start server connection : get_dai_client -----
<class 'driverlessai._core.Client'>
DAIバージョン: 1.10.1.3
*************** データの取得 ***************
----- start get data : get_dataset -----
Uploaded data name : key >>  {'walmart_ts_6_fcst_grp_train.csv': '4d2ad106-9b86-11ec-8e63-0242ac110002', 'walmart_ts_6_fcst_grp_test.csv': '4d2a15fe-9b86-11ec-8e63-0242ac110002', 'kaggle_train_index.csv': '7477e2b4-9369-11ec-ae6e-0242ac110002', 'kaggle_train2_int.csv': '757ae702-9368-11ec-ae6e-0242ac110002', 'BostonHousing.csv': '653b25c2-91de-11ec-8bed-0242ac110002'}
Data is already uploaded in DAI
<class 'driverlessai._datasets.Dataset'>
Dataサイズ(byte): 57768
Dataサイズ(mega byte): 0.05509185791015625
Data shape: (506, 14)
*************** Experimentの実施 ***************
----- start experiment : get_experiment -----
Experiment launched at: http://3.82.142.224/#/experiment?key=7dc555a4-9de6-11ec-a1da-0242ac110002
Complete 100.00% - Status: Complete                     

### 実験情報

In [42]:
log_datasize_mb = ds.file_size/1024**2     # 元データサイズ（MB）
log_n_observation = ds.shape[0]    # オブザベーション数
log_n_features = ds.shape[1] - len(drop_columns) - 1    # 特徴量数
log_duration_min = ex.run_duration/60    # 実行時間（分）
log_expsize_mb = ex.size/1024**2     # Experimentサイズ（MB）
log_accuracy_time_interpretability = ex.settings['accuracy'], ex.settings['time'], ex.settings['interpretability']   # accuracy, time, interpretability

print(log_datasize_mb)
print(log_n_observation)
print(log_n_features)
print(log_duration_min)
print(log_expsize_mb)
print(log_accuracy_time_interpretability)

0.05509185791015625
506
13
4.895818114280701
1062.600456237793
(7, 2, 8)


## 他サーバ（GPU）

In [19]:
# Driverless AIサーバー情報
dai_address = 'http://44.201.226.2'
dai_password = idpass['pass11013gpu']
# データ情報
data_name = 'BostonHousing.csv'
s3url = 's3://h2oai-jp-public/sample_data/boston_house/BostonHousing.csv'   # DAIにアップされてない場合の取得先S3
# Experiment設定
target_column = 'MEDV'
task = 'regression'    # 'regression', 'classification', or 'unsupervised'
drop_columns = []    # List[str] 説明変数として利用しないカラム
#drop_columns = None
enable_gpus = True

In [21]:
print('*************** DAIへ接続 ***************')
dai = get_dai_client(daiaddress=dai_address, daipassword=dai_password)
print(type(dai))
print('DAIバージョン: {}'.format(dai.server.version))


print('*************** データの取得 ***************')
ds = get_dataset(daiobj=dai, dataname=data_name, dataurl=s3url) 

print(type(ds))
print('Dataサイズ(byte): {}'.format(ds.file_size))
print('Dataサイズ(mega byte): {}'.format(ds.file_size/1024**2))
print('Data shape: {}'.format(ds.shape))


print('*************** Experimentの実施 ***************')
ex = get_experiment(daiobj=dai, dataobj=ds, 
                    target_column=target_column, task=task, drop_columns=drop_columns, enable_gpus=enable_gpus)

print(type(ds))
print('学習時間（sec）：{}'.format(ex.run_duration))
print('学習時間（min）：{}'.format(ex.run_duration/60))
print('Experimentサイズ（byte）：{}'.format(ex.size))
print('Experimentサイズ（mega byte）：{}'.format(ex.size/1024**2))
print('********** Experiment Summary **********')
ex.summary()

*************** DAIへ接続 ***************
----- start server connection : get_dai_client -----
<class 'driverlessai._core.Client'>
DAIバージョン: 1.10.1.3
*************** データの取得 ***************
----- start get data : get_dataset -----
Uploaded data name : key >>  {'BostonHousing.csv': '6db78e82-91f9-11ec-a31a-0242ac110002'}
Data is already uploaded in DAI
<class 'driverlessai._datasets.Dataset'>
Dataサイズ(byte): 57768
Dataサイズ(mega byte): 0.05509185791015625
Data shape: (506, 14)
*************** Experimentの実施 ***************
----- start experiment : get_experiment -----
Experiment launched at: http://44.201.226.2/#/experiment?key=a920f88a-91fb-11ec-a31a-0242ac110002
Complete 100.00% - Status: Complete                                                       
<class 'driverlessai._datasets.Dataset'>
学習時間（sec）：554.839200258255
学習時間（min）：9.24732000430425
Experimentサイズ（byte）：1117560290
Experimentサイズ（mega byte）：1065.7885456085205
********** Experiment Summary **********
Status: Complete
Experiment: sumuw